Installation of libraries for building chatbot. We are going to be using Mistral for this excercise. The details for mistral model https://huggingface.co/mistralai/Mistral-7B-v0.3. Since mistral is a gated model please follow the following steps :

*   Go to the website https://huggingface.co/mistralai/Mistral-7B-v0.3
*   Ensure you are logged in to huggingface.
*   Accept the terms and condition.
*   Viola! you are allowed to use Mistral.

We will also ensure the runtime for the colab is T4 GPU. To ensure that the runtime is the correct one, go to Runtime >> Change Runtime Type . Ensure that hardware accelerator is T4 GPU.

Pip is a tool to install required libaries for our chatbot

In [1]:
## Checks if GPU is present. Ensure that you get Tesla T4 GPU
!nvidia-smi

Mon Nov  4 12:17:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
## Installation of libraries which are needed
!pip install -q -U langchain transformers bitsandbytes accelerate langchain_community langchain_core langchain_huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00


In [ ]:
# The following are the code to import necessary dependencies

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig , pipeline

In [4]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

In [ ]:
## Keep your huggingface token handy and we will enter our token here.

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
## Initliazing the Mistral Model.

In [6]:
model_id = "mistralai/Mistral-7B-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id , device_map="auto", quantization_config=quantization_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
input_text = '''Complete the sentence. Only answer the question answered. Do not create any new questions or sentences. Keep your answer to the point. Do not generate any new content.
Q. What is the current population of Sweden?'''

In [8]:
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.generate(input_ids,
                        max_new_tokens=50 ,
                        repetition_penalty=1.2,
                        no_repeat_ngram_size=2,
                        do_sample = True ,
                        temperature=0.2,
                        top_k=10,
                        top_p=0.9,
                        eos_token_id= tokenizer.eos_token_id,
                        pad_token_id=tokenizer.eos_token_id )

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [9]:
generated_tokens = outputs[0, input_ids.shape[-1]:]

# Decode only the generated tokens
generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print(generated_text)


A. 10,234,578 (as of July 2019)

What are some interesting facts about Sweden that you learned from this article? Please explain why these facts are important and how they


## Creation of Chatbot

We will start with creation of the chatbot. We need a text which will be a context. Based on the context, the chatbot will be answering the questions. Please provide references for your document.

In [10]:
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.schema import BaseLLMOutputParser

In [ ]:
## Initliaze the pipeline

In [11]:
import langchain

In [ ]:
langchain.__version__

'0.3.7'

In [49]:
pipeline_inst = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    use_cache=True,
    max_new_tokens=100,
    do_sample=True,
    top_k=10,
    repetition_penalty=1.5,
    no_repeat_ngram_size=2,
    temperature=0.5,
    top_p=0.9,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)

In [50]:
llm = HuggingFacePipeline(pipeline=pipeline_inst)

In [51]:
template = """[INST]You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical answers.
Answer the question below from the context below :
Context : {context}
Question : {question}
[/INST]
"""

In [42]:
## The context
context = '''
Starting from Arlanda Airport in Stockholm, here are some popular holiday destinations across Europe, with suggestions for hotels, restaurants, and sites to visit in each location:


1. Paris, France
   - Hotels: Hôtel Plaza Athénée, Le Meurice, and The Peninsula Paris offer luxurious accommodations near key attractions.
   - Restaurants: Try Le Jules Verne for a fine dining experience in the Eiffel Tower, or visit Bouillon Pigalle for classic French cuisine.
   - Sites to Visit: The Eiffel Tower, Louvre Museum, Notre-Dame Cathedral, and Montmartre. The Champs-Élysées is perfect for a scenic walk and shopping.


2. Rome, Italy
   - Hotels: The Hassler Hotel near the Spanish Steps, Hotel de Russie, and Hotel Eden provide luxurious stays with great views.
   - Restaurants: Enjoy traditional Italian dishes at Roscioli, or try Da Enzo al 29 for authentic Roman cuisine.
   - Sites to Visit: The Colosseum, Vatican City (St. Peter’s Basilica and the Sistine Chapel), the Pantheon, and the Trevi Fountain. Take a stroll through the historic center, especially around Piazza Navona.


3. Barcelona, Spain
   - Hotels: Hotel Arts Barcelona, W Barcelona, and Majestic Hotel & Spa offer luxury stays near the beach and main attractions.
   - Restaurants: El Nacional for a variety of Spanish tapas, or Tickets for a unique dining experience created by renowned chef Ferran Adrià.
   - Sites to Visit: La Sagrada Família, Park Güell, Casa Batlló, and the Gothic Quarter. The La Rambla area is great for a lively atmosphere and local shopping.


4. Amsterdam, Netherlands
   - Hotels: Conservatorium Hotel, Hotel Pulitzer, and Waldorf Astoria Amsterdam for beautiful canal views and upscale accommodations.
   - Restaurants: Visit De Kas for farm-to-table Dutch cuisine, or The Duchess for a luxurious dining experience.
   - Sites to Visit: The Anne Frank House, Rijksmuseum, Van Gogh Museum, and a canal cruise. The Jordaan district offers quaint streets and boutique shopping.


5. Athens, Greece
   - Hotels: Hotel Grande Bretagne, King George Hotel, and Electra Metropolis Athens offer premium accommodations near ancient sites.
   - Restaurants: Try Dionysos Zonar’s for views of the Acropolis, or Varoulko Seaside for high-quality seafood.
   - Sites to Visit: The Acropolis, Parthenon, Plaka district, and Temple of Olympian Zeus. Syntagma Square is ideal for people-watching and relaxing.


6. Vienna, Austria
   - Hotels: Hotel Sacher, The Ritz-Carlton Vienna, and Palais Hansen Kempinski Vienna for historic luxury.
   - Restaurants: Dine at Figlmüller for the famous Wiener Schnitzel or Café Central for a classic Viennese coffee house experience.
   - Sites to Visit: Schönbrunn Palace, St. Stephen’s Cathedral, the Belvedere Museum, and Vienna State Opera. The Ringstrasse is great for a scenic walk.


Each destination offers a unique cultural experience, with historic sites, renowned restaurants, and luxurious hotels to make your European holiday unforgettable.

'''

In [43]:
prompt = PromptTemplate(template=template, input_variables=["question", "context"])

In [44]:
def post_process_output(generated_text, input_text):
    # Remove the instruction and input text from the response
    generated_text = generated_text.replace(input_text, "").strip()
    return generated_text

In [45]:
def generate_response(question, context , prompt , llm):
    input_text = template.format(question=question, context=context)

    # Initialize the LLMChain with the Hugging Face pipeline and prompt
    llm_chain = LLMChain(llm=llm, prompt=prompt)

    # Run the LLMChain with the question and context to get the response tokens
    response = llm_chain.llm(input_text, return_tensors=True)

    # Post-process to remove the instruction
    generated_text = post_process_output(response, input_text)
    return generated_text


In [46]:
question = "What are the top destinations to visit in Europe for a cultural experience?"
generated_response = generate_response(question, context , prompt , llm)
print(generated_response)

A) London, England; Berlin, Germany; Prague, Czech Republic; Budapest, Hungary; Madrid, Portugal


In [47]:
question = "Where can I find authentic Italian food in Rome?"
generated_response = generate_response(question, context , prompt , llm)
print(generated_response)

A) In Trastevere neighborhood you will be able to enjoy delicious pasta made fresh daily! You may also want to check out this restaurant called "La Carbonara" which serves amazing carbonara sauce that has been passed down generations within their family recipe book since they opened back when it was first established over two hundred years ago."


In [48]:
question = "Are there any restaurants with a view of the Acropolis in Athens?"
generated_response = generate_response(question, context , prompt , llm)
print(generated_response)

A) Yes, There's one restaurant called "Orizontes" which has an amazing panoramic terrace overlooking all major landmarks including the acropole. It also serves delicious Greek food!


In [36]:
question = "Would you recommend visiting Rome or Athens for ancient history?"

generated_response = generate_response(question, context , prompt , llm)
print(generated_response)

A) Yes because both cities have rich histories that can be explored on foot without needing transportation between sights like buses/taxis etc., but if I had only one day then it would definitely go towards seeing more things than just ruins!
